### Tutorial: How to Use the Given Code

This notebook is a guide to using the code provided in our package. This example illustrates how we have used our package to analyze our microtubule catastrophe data. 

The first thing we would need to do is to download the package onto your computer. After this, you can simply import the package and any other packages you will need using import statements as exemplified below:

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
# Using alias for packages can reduce having to write out the entire name every single time you call them
import pandas as pd
import holoviews as hv
import MCAT_pkg as mc 

You can find out more information about a package or function by adding "?" or "??" behind its name:

In [2]:
mc?

Type:        module
String form: <module 'MCAT_pkg' (namespace)>
Docstring:   <no docstring>


After we import our package, we will need to load in our data. 

In [3]:
mc??

Type:        module
String form: <module 'MCAT_pkg' (namespace)>
Docstring:   <no docstring>


In [4]:
# Identify the location that the data file is in  
data_path = "data/gardner_mt_catastrophe_only_tubulin.csv"

# Read in the file as df
df = pd.read_csv(data_path, comment = "#")

Let's take a look at the first 5 rows of the data using the command:

In [13]:
df.head()

,12 uM,7 uM,9 uM,10 uM,14 uM
0,25.000,35.0,25.0,50.0,60.0
1,40.000,45.0,40.0,60.0,75.0
2,40.000,50.0,40.0,60.0,75.0
3,45.429,50.0,45.0,75.0,85.0
4,50.000,55.0,50.0,75.0,115.0


We would like to parse the dataframe into a tidy format. For more information about tidy data, go to the link [here.](https://bebi103a.github.io/lessons/06/tidy_data.html)

In [14]:
# Use pandas.melt() to unpivot the dataframe to get Concentration and Catastrophe time into separate columns
df_parsed = pd.melt(df, value_vars=list(df.columns.values), 
                    var_name = "Concentration (uM)", 
                    value_name = "Time to catastrophe (s)")
# Remove missing values
df_parsed = df_parsed.dropna()

Let's take a look at the dataframe.

In [15]:
df_parsed.head()

,Concentration (uM),Time to catastrophe (s)
0,12 uM,25.000
1,12 uM,40.000
2,12 uM,40.000
3,12 uM,45.429
4,12 uM,50.000


Next, we would like to define a categorical color palette to use for different tubulin concentrations

In [16]:
colors = mc.bokeh.palettes.Dark2[5]

AttributeError: module 'MCAT_pkg' has no attribute 'bokeh'

We would now perform exploratory analysis on the dataset using an ECDF, since they visualize the cumulative density function (CDF). For more information, visit the link [here.](https://bebi103a.github.io/lessons/06/visualizing_distributions.html)

In [ ]:
p = mc.iqplot.ecdf(df_parsed, q = "Time to catastrophe (s)", cats = "Concentration (uM)", order=['7 uM', '9 uM', '10 uM', '12 uM', '14 uM'])
p.title.text = "ECDF of Catastrophe times by Concentration"
mc.bokeh.io.show(p)

From the ECDF, we can see that the tubulin concentration 14uM has the longest overall time to catastrophe, followed by 12 uM, 10uM, and 7uM/9uM. We can see a general trend that higher tubulin concentrations tend to have longer catastrophe times. 

We now want to see what the individual tubulin concentrations would look like as stripbox plots. 

In [ ]:
p_strip = mc.iqplot.stripbox(df_parsed, q = "Time to catastrophe (s)", cats = "Concentration (uM)", order=['7 uM', '9 uM', '10 uM', '12 uM', '14 uM'])
p_strip.title.text = "Stripbox Plot of Catastrophe Times, separated by Concentration"
mc.bokeh.io.show(p_strip)

Here, we can see a similar trend that higher tubulin concentrations tend to have longer catastrophe times. One thing to note, however, is that catastrophe times of tubulin concentration 12uM has more outliers, which can influence/bias the mean. 

Now, we would like to compare whether a Gamma distribution or an exponential distribution is a better model for this data. For simplicity, we will first perform the tests on tubulin concentration 12uM and then the rest of the concentrations once we determine which model is better. 

We need to pull out the data for catastrophe times for a tubulin concentration of 12 uM.

In [ ]:
# Generate array of times to catastrophe for 12 uM concentration of tubulin
conc_12_times = df_parsed.loc[df_parsed["Concentration (uM)"] == '12 uM', 'Time to catastrophe (s)'].values

#### Gamma Distribution Model
We will first generate parameter estimates according to the Gamma distribution. You can look at the functions within the MCAT_pgk folder. 

In [ ]:
mc.mle_iid_gamma(conc_12_times)

Perform maximum likelihood estimates for parameters for i.i.d. gamma measurements, parametrized by alpha, b=1/beta

In [ ]:
mc.MLE_analysis.log_like_iid_gamma_log_params(mc.mle_iid_gamma(conc_12_times), conc_12_times)

We will draw bootstrap replicates of our MLE. 

In [ ]:
gamma_bs_reps = mc.draw_bs_reps(conc_12_times, mc.mle_iid_gamma, size=100)

This will give us the MLE estimate.

In [ ]:
gamma_mle = mc.np.mean(gamma_bs_reps, axis = 0)

#### Exponential Distribution Model

We will move on to generate parameter estimates according to the exponential distribution.

In [ ]:
# For example sake, we will only be bootstrapping 100 replicates 
# instead of 10000 that we used in our analysis
exp_bs_reps = mc.draw_bs_reps(conc_12_times, mc.mle_iid_exp, size=100)

In [ ]:
exp_mle = mc.np.mean(exp_bs_reps, axis = 0)

We want to get the AIC values for each distribution 

In [ ]:
# Exponential distribution
mc.AIC(exp_mle, mc.log_like_iid_exp_log_params, conc_12_times)

In [ ]:
# Gamma distribution 
mc.AIC(gamma_mle, mc.MLE_analysis.log_like_iid_gamma_log_params, conc_12_times)

We can now plot the QQ plot for the exponential model. 

In [ ]:
p = mc.QQ_plot(conc_12_times, mc.gen_exponential, exp_mle, size = 10000, 
               axis_label = "Time to catastrophe (s)", title = "Exponential QQ-Plot")
mc.bokeh.io.show(p)

The QQ plot for the gamma model.

In [ ]:
p2 = mc.QQ_plot(conc_12_times, mc.gen_gamma, gamma_mle, size = 10000, 
                axis_label = "Time to catastrophe (s)", title = "Gamma QQ-Plot")
mc.bokeh.io.show(p2)

We would also like to compare the ECDF of theoretical distribution to experimental for each model. 

In [ ]:
# Exponential distribution 
p3 = mc.predictive_ecdf(conc_12_times, mc.gen_exponential, exp_mle, title = 'Exponential Distribution ECDF')
mc.bokeh.io.show(p3)

In [ ]:
# Gamma distribution 
p4 = mc.predictive_ecdf(conc_12_times, mc.gen_gamma, gamma_mle, title = 'Gamma Distribution ECDF')
mc.bokeh.io.show(p4)

Since the AIC of the exponential distribution (9327.391952721562) is higher than that of the gamma distribution (9278.358353623804), the exponential model is preferred. From the QQ-plots, we can see that there is more separation of the generative quantiles from the observed quantiles in the gamma distribution model than the exponential distribution model. As a result, based on AIC (Akaike information criterion) and QQ plot, the exponential distribution model is better model. 

### Parameter Estimates for the Other Tubulin Concentrations  

Now that we have a preferred model, we will obtain parameter estimates for the other tubulin concentrations using the exponential model. 

In [ ]:
# Get unique concentrations and take out 12uM
all_concentrations = df_parsed["Concentration (uM)"].unique()
concentrations = mc.np.delete(all_concentrations, 0)
all_other_bs_reps = []
all_other_conf_int = []
all_other_mean = []

In [ ]:
# Generate bootstrap replicates for the other tubulin concentrations using the exponential model
# Generate array of times to catastrophe for other concentrations of tubulin
for conc in concentrations:
    conc_times = df_parsed.loc[df_parsed["Concentration (uM)"] == conc, 
                                  'Time to catastrophe (s)'].values
    bs_reps = mc.draw_bs_reps(conc_times, mc.mle_iid_exp, size=100)
    all_other_bs_reps.append(bs_reps)
    mean = mc.np.mean(bs_reps, axis = 0)
    all_other_mean.append(mean)
    conf_int = mc.np.percentile(bs_reps, [2.5, 97.5], axis=0)
    all_other_conf_int.append(conf_int)
    print('The MLE for number of arrivals for a catastrophe to occur (b_1) and difference in b_1 and b_2 (delta_b) is respectively {} with a 95% confidence interval of \n{}\n.'
          .format(mean, conf_int))

We can now plot the bootstrap replicates of all the tubulin concentrations. 

In [ ]:
points = hv.Points([]).opts(width=500, height=300)
all_conc = mc.np.append(concentrations, '12 uM')
all_other_bs_reps.append(exp_bs_reps)
for i in range(len(all_conc)):
    # Package replicates in data frame for plotting
    df_res = pd.DataFrame(data=all_other_bs_reps[i], columns=["b_1*", "delta_b*"])
    points *= hv.Points(
        data=df_res,
        kdims=["b_1*", "delta_b*"],
        label = 'Concentration {}'.format(all_conc[i]),
    ).opts(
        size=2.5,
        alpha=2.5,
        color=colors[i],
        width=700, 
        height=500,
        legend_position='right',
        legend_offset=(0, 290)
    )
points

points.opts(title='Plot of 𝛽1 and Δ𝛽',xlabel='beta (1/s)', ylabel='Delta_beta (1/s)', width=700, height=500)

From the plot of $\beta_1$ and $\Delta\beta$ of the different tubulin concentrations, we can see that higher tubulin concentrations seem to have lower $\beta_1$ values (1/(time to first catastrophe)). All the mean $\beta_1$values for the different tubulin concentrations, however, are all on the order of 1e-3. In general, by looking at the values of the parameters, we see that the $\Delta\beta$ values are on the order of 0 to 1e-4 or 1e-5, which means that $\beta_1$ and $\beta_2$ are often the same or are very similar in value, indicating that catastrophe times one after another are often very similar and may be dependent. 